<a href="https://colab.research.google.com/github/richardlin01/CodeReview/blob/main/GiaiThich.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Khai báo các thư viện

- Khai báo các thư viện cần thiết cho dự án, bao gồm FirebaseESP32, HardwareSerial, và WiFi.
- Các thư viện này cho phép kết nối và giao tiếp với Firebase, sử dụng cổng serial và thiết lập kết nối WiFi.

In [ ]:
#include<FirebaseESP32.h>
#include<HardwareSerial.h>
#include<WiFi.h>

- define Macro ID và Pass Wifi
- define auth & host của Firebase

Định nghĩa WiFi và Firebase Credentials: Xác định thông tin đăng nhập WiFi (SSID và mật khẩu) và thông tin xác thực Firebase (FIREBASE_AUTH) cần thiết để kết nối đến mạng và cơ sở dữ liệu Firebase.

In [ ]:
// WiFi credentials
#define WIFI_SSID "TB1_ACTVN"
#define WIFI_PASSWORD "1234567@9"
// Firebase credentials
#define FIREBASE_AUTH "P1tLoliGeLjFXgjCvJWKLqTTV4UAoWKR3FIuUlFg"
#define FIREBASE_HOST "https://iotzigbee-546dd-default-rtdb.firebaseio.com/"
// Firebase database instance

Khai báo các biến trạng thái, nhiệt độ, độ ẩm, khí gas, biến thời gian hiện tại và trước đó.

In [ ]:
String path = "/SmartHomeZigbee";
FirebaseData firebaseData;
HardwareSerial zigbee(2);
String data_receiv, node, data_tx, tmp1, humi1, mq2_s, tt_den, tt_quat,s = "";
int tt ;
float temperature1;
float humidity1;
int mq2;
unsigned long curr_time, prev_time;

Chương trình chính:
- Setup baudrate truyền của Serial và của Zigbee
- Connect Wifi
- setString đẩy dữ liệu lên Firebase 2 trường LED và FAN

Trong hàm setup():

- Bắt đầu giao tiếp Serial với baud rate là 9600 và cổng serial Zigbee với baud

rate là 115200.

- Kết nối đến mạng WiFi sử dụng thông tin đăng nhập đã được định nghĩa. Một vòng lặp while được sử dụng để kiểm tra xem kết nối WiFi đã thành công hay chưa.

- Khởi tạo kết nối đến Firebase Database bằng cách sử dụng Firebase.begin() với thông tin xác thực Firebase và đường dẫn cơ sở dữ liệu Firebase.

- Đặt giá trị mặc định cho các nút "LED" và "FAN" trong cơ sở dữ liệu Firebase bằng Firebase.setString().

In [ ]:
void setup() {

  // Start serial communication
  Serial.begin(9600);
  zigbee.begin(115200, SERIAL_8N1, 16, 17);
  // Connect to WiFi network
  WiFi.begin(WIFI_SSID, WIFI_PASSWORD);
  while (WiFi.status() != WL_CONNECTED) {
    delay(10);
    Serial.println("Connecting to WiFi...");
  }
  // Initialize Firebase
  Firebase.begin(FIREBASE_HOST, FIREBASE_AUTH);
  Serial.println("Connected to firebase");
  Firebase.setString(firebaseData, path + "/LED", "off");
  Firebase.setString(firebaseData, path + "/FAN", "off");
}

- Vòng loop() của chương trình.

In [ ]:
void loop() {
  if (zigbee.available() > 0) {
    data_receiv = zigbee.readStringUntil('\n');
    delay(1);
  }
  curr_time = millis();
  if (curr_time - prev_time > 200) {
    prev_time = curr_time;
    tt++;
    if (tt < 10) {
      //data_tx = firebaseData.intData();
      Firebase.getString(firebaseData, path + "/LED");
      tt_den = firebaseData.stringData();
      Firebase.getString(firebaseData, path + "/FAN");
      tt_quat = firebaseData.stringData();
      //tt_den = firebaseData.stringData();   // quat
      s = s + tt_den;
      Serial.println(s);
      data_tx = "10/"  + s + "/" + tt_quat;    // 10/on
      zigbee.println(data_tx);
      s = "";
      node = getValue(data_receiv, '/', 0);
      if (node == "node1") {
        tmp1 = getValue(data_receiv, '/', 1);
        humi1 = getValue(data_receiv, '/', 2);
        temperature1 = tmp1.toFloat();
        humidity1 = humi1.toFloat();
        Firebase.setFloat(firebaseData, path + "/TEMPERATURE", temperature1);
        Firebase.setFloat(firebaseData, path + "/HUMIDITY", humidity1);
        Serial.print("Temp:");
        Serial.print(temperature1);
        Serial.print("---");
        Serial.print("Humi:");
        Serial.print(humidity1);
        Serial.println();
      }
    }
    if (tt < 20 && tt > 10) {
      data_tx = "20/"  + String(tt_den);    // tt_quat
      zigbee.println(data_tx);
      node = getValue(data_receiv, '/', 0);
      if (node == "node2") {
        mq2_s = getValue(data_receiv, '/', 1);
        mq2 = mq2_s.toInt();
        Firebase.setFloat(firebaseData, path + "/MQ2", mq2);
        Serial.print("MQ2:");
        Serial.print(mq2_s);
        Serial.println();
      }
    }
    if (tt > 20) {
      tt = 0;
    }
  }
}
String getValue(String data, char separator, int index) {
  int found = 0;
  int strIndex[] = { 0, -1 };
  int maxIndex = data.length() - 1;
  for (int i = 0; i <= maxIndex && found <= index; i++) {
    if (data.charAt(i) == separator || i == maxIndex) {
      found++;
      strIndex[0] = strIndex[1] + 1;
      strIndex[1] = (i == maxIndex) ? i + 1 : i;
    }
  }
  return found > index ? data.substring(strIndex[0], strIndex[1]) : "";
}

Giải thích vòng Loop()

Hàm loop():

Đây là hàm chính chạy trong vòng lặp vô hạn và thực hiện các công việc:

- Kiểm tra xem có dữ liệu mới từ module Zigbee thông qua zigbee.available(). Nếu có dữ liệu, nó sẽ được đọc vào biến data_receiv.

- Tạo một biến curr_time để theo dõi thời gian hiện tại và biến prev_time để kiểm tra khoảng thời gian giữa các lần lặp.

- Sử dụng biến tt để theo dõi số lần lặp và thực hiện các hành động khác nhau dựa trên giá trị của tt.

- Đọc giá trị "LED" và "FAN" từ cơ sở dữ liệu Firebase bằng Firebase.getString() và tạo chuỗi data_tx để gửi dữ liệu đến module Zigbee.

- Đọc dữ liệu từ module Zigbee và cập nhật cơ sở dữ liệu Firebase với dữ liệu từ cảm biến "node1" (nhiệt độ và độ ẩm) hoặc "node2" (dữ liệu từ cảm biến MQ2).

- Sử dụng hàm getValue() để phân tách dữ liệu nhận được từ module Zigbee theo ký tự '/' và lấy các giá trị cụ thể.

- Cuối cùng, kiểm tra giá trị của tt và đặt lại nó thành 0 nếu nó vượt quá 20 để quản lý các lần lặp.

In [ ]:
if (zigbee.available() > 0) {
    data_receiv = zigbee.readStringUntil('\n');
    delay(1);
  }
  curr_time = millis();
  if (curr_time - prev_time > 200) {
    prev_time = curr_time;
    tt++;
    if (tt < 10) {
      //data_tx = firebaseData.intData();
      Firebase.getString(firebaseData, path + "/LED");
      tt_den = firebaseData.stringData();
      Firebase.getString(firebaseData, path + "/FAN");
      tt_quat = firebaseData.stringData();
      //tt_den = firebaseData.stringData();   // quat
      s = s + tt_den;
      Serial.println(s);
      data_tx = "10/"  + s + "/" + tt_quat;    // 10/on
      zigbee.println(data_tx);
      s = "";
      node = getValue(data_receiv, '/', 0);
      if (node == "node1") {
        tmp1 = getValue(data_receiv, '/', 1);
        humi1 = getValue(data_receiv, '/', 2);
        temperature1 = tmp1.toFloat();
        humidity1 = humi1.toFloat();
        Firebase.setFloat(firebaseData, path + "/TEMPERATURE", temperature1);
        Firebase.setFloat(firebaseData, path + "/HUMIDITY", humidity1);
        Serial.print("Temp:");
        Serial.print(temperature1);
        Serial.print("---");
        Serial.print("Humi:");
        Serial.print(humidity1);
        Serial.println();
      }
    }
    if (tt < 20 && tt > 10) {
      data_tx = "20/"  + String(tt_den);    // tt_quat
      zigbee.println(data_tx);
      node = getValue(data_receiv, '/', 0);
      if (node == "node2") {
        mq2_s = getValue(data_receiv, '/', 1);
        mq2 = mq2_s.toInt();
        Firebase.setFloat(firebaseData, path + "/MQ2", mq2);
        Serial.print("MQ2:");
        Serial.print(mq2_s);
        Serial.println();
      }
    }
    if (tt > 20) {
      tt = 0;
    }
  }
}

Hàm getValue():

- Hàm này được sử dụng để trích xuất giá trị từ một chuỗi dựa trên ký tự ngăn cách.
-Hàm này có tham số đầu vào là data (chuỗi cần xử lý), separator (ký tự dùng để phân tách giá trị), và index (chỉ định giá trị cần lấy).
- Hàm này sẽ trả về giá trị tại vị trí index trong chuỗi sau khi đã tách.


In [ ]:
String getValue(String data, char separator, int index) {
  int found = 0;
  int strIndex[] = { 0, -1 };
  int maxIndex = data.length() - 1;
  for (int i = 0; i <= maxIndex && found <= index; i++) {
    if (data.charAt(i) == separator || i == maxIndex) {
      found++;
      strIndex[0] = strIndex[1] + 1;
      strIndex[1] = (i == maxIndex) ? i + 1 : i;
    }
  }
  return found > index ? data.substring(strIndex[0], strIndex[1]) : "";
}


Tổng kết: Đoạn code này tổ chức việc thu thập dữ liệu từ cảm biến và truyền chúng đến cơ sở dữ liệu Firebase thông qua Zigbee. Nó cũng đảm bảo rằng dữ liệu được đọc và xử lý một cách đúng đắn nhất và được đồng bộ hóa với Firebase.